In [0]:
#!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [7]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  Y = df['prices_amountmin'].values
  
  scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [12]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [14]:
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)
              

(-57.47223572384038, 4.328288468270897)

In [15]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [18]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-57.31783843165656, 4.181246596160967)

In [19]:
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [21]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [24]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) =='nan': return output_dict
  
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [32]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [33]:
keys = set()
df['features_parsed'].map(lambda x:keys.update(x.keys()))
len(keys)


476

In [36]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats:  feats[key] if key in feats else np.nan)

In [38]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_ul safety listing', 'feat_type 2', 'feat_is lined',
       'feat_item weight unit of measure', 'feat_chain/necklace length (in.)',
       'feat_waterproof', 'feat_lining', 'feat_has paper wood', 'feat_sku',
       'feat_ring size'],
      dtype='object', length=526)

In [39]:
df[df['feat_athlete'].isnull()].shape

(18272, 526)

In [40]:
df.shape[0]

18280

In [41]:
df[False ==df['feat_athlete'].isnull()].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False ==df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [48]:
{k:v for k,v in keys_stat.items() if v >30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df ['feat_brand'].factorize()[0]
df['feat_color_cat'] = df ['feat_color'].factorize()[0]
df['feat_gender_cat'] = df ['feat_gender'].factorize()[0]
df['feat_manufacturer_part_number_cat'] = df ['feat_manufacturer_part_number'].factorize()[0]
df['feat_material_cat'] = df ['feat_material'].factorize()[0]

In [50]:
df[df.brand !=df.feat_brand].shape

(18228, 531)

In [51]:
df[df.brand ==df.feat_brand].shape

(52, 531)

In [54]:
df[df.brand ==df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [55]:
df[df.brand !=df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [57]:
df['brand']=df['brand'].map(lambda x: str(x).lower())
df[df.brand !=df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [58]:
df[df.brand ==df.feat_brand].shape

(8846, 531)

In [0]:
feats = ['']

In [60]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.318845340606764, 4.146665673209246)

In [62]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer_part_number_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.13250890187368, 4.23132310908335)

In [64]:
X = df[feats].values
Y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth = 5, n_estimators =100, random_state=0)
m.fit(X, Y)

perm =PermutationImportance(m, random_state=1).fit(X, Y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2568 ± 0.0106,brand_cat
0.1062 ± 0.0094,feat_material_cat
0.0426 ± 0.0044,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0023 ± 0.0004,feat_manufacturer_part_number_cat


In [66]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.24046214157587, 4.182277361502224)

In [67]:
X = df[feats].values
Y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth = 5, n_estimators =100, random_state=0)
m.fit(X, Y)

perm =PermutationImportance(m, random_state=1).fit(X, Y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2595 ± 0.0080,brand_cat
0.1099 ± 0.0134,feat_material_cat
0.0435 ± 0.0055,feat_gender_cat
0.0224 ± 0.0007,feat_brand_cat


In [68]:
df['brand'].value_counts(normalize=True)

nike                                  0.097210
puma                                  0.033315
ralph lauren                          0.028775
vans                                  0.021116
new balance                           0.020295
                                        ...   
suunto                                0.000055
co7de                                 0.000055
coolomg                               0.000055
signature by levi strauss &amp; co    0.000055
otz shoes                             0.000055
Name: brand, Length: 1732, dtype: float64

In [71]:
df[df['brand'] =='nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df['feat_brand_cat'] = df ['feat_brand'].factorize()[0]
df['feat_color_cat'] = df ['feat_color'].factorize()[0]
df['feat_gender_cat'] = df ['feat_gender'].factorize()[0]
df['feat_manufacturer_part_number_cat'] = df ['feat_manufacturer_part_number'].factorize()[0]
df['feat_material_cat'] = df ['feat_material'].factorize()[0]

df['feat_sport_cat'] = df ['feat_sport'].factorize()[0]
df['feat_style_cat'] = df ['feat_style'].factorize()[0]


In [73]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.05815420087178, 4.239718182630683)

In [74]:
X = df[feats].values
Y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth = 5, n_estimators =100, random_state=0)
m.fit(X, Y)

perm =PermutationImportance(m, random_state=1).fit(X, Y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2608 ± 0.0111,brand_cat
0.1046 ± 0.0083,feat_material_cat
0.0455 ± 0.0046,feat_gender_cat
0.0216 ± 0.0012,feat_brand_cat
0.0070 ± 0.0013,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [75]:
df[df['brand'] =='nike'].features_parsed.sample(5).values

array([{'style': 'athletic sneakers', 'condition': 'new with box'},
       {'season': 'all-season', 'shoe size': '11', 'color': 'pink', 'model': '599264 600', 'casual & dress shoe style': 'basketball', 'gender': 'men', 'brand': 'nike', 'age group': 'adult', 'shoe width': 'medium (d, m)', 'material': 'synthetic', 'size': '11', 'shoe category': 'mens shoes', 'manufacturer part number': '599264 600'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'white/university blue/pure platinum', 'casual & dress shoe style': 'running shoes', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'sport': 'soccer', 'main color': 'multi-colored'},
       {'gender': 'men', 'size': 'l', 'brand': 'nike', 'fabric content': '100% cotton', 'color': 'white/black'}],
      dtype=object)

In [0]:
df['feat_brand_cat'] = df ['feat_brand'].factorize()[0]
df['feat_color_cat'] = df ['feat_color'].factorize()[0]
df['feat_gender_cat'] = df ['feat_gender'].factorize()[0]
df['feat_manufacturer_part_number_cat'] = df ['feat_manufacturer_part_number'].factorize()[0]
df['feat_material_cat'] = df ['feat_material'].factorize()[0]

df['feat_sport_cat'] = df ['feat_sport'].factorize()[0]
df['feat_style_cat'] = df ['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [78]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_catalog',
 'feat_fabrication',
 'feat_location - country',
 'feat_certifications and listings',
 'feat_clothing category',
 'feat_shoe category',
 'feat_recommended location',
 'feat_multi pack indicator',
 'feat_location - city/state',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer_part_number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_item color_cat',
 'feat_watch case shape_cat',
 'feat_high visibility (ansi compliant)_cat',
 'feat_uv protection_cat',
 'feat_age segment_cat',
 'feat_contained battery type_cat',
 'feat_sku number_cat',
 'feat_age start_cat',
 'feat_characteristics_cat',
 'feat_free item_cat',
 'feat_to fit_cat',
 'feat_country//region of manufacture_cat',
 'feat_airport friendly_cat',
 'feat_package_cat',
 'feat_footwear type_cat',
 'feat_clothing product type_cat',
 'feat_fishing_cat',
 'feat_sizebridge_cat',
 'feat_charge time (minutes)_cat',
 'feat_shoe width_cat',
 'f

In [82]:
feats = ['brand_cat','feat_shape_cat', 'feat_metal type_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.27533077579017, 4.232911284629363)

In [83]:
X = df[feats].values
Y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth = 5, n_estimators =100, random_state=0)
m.fit(X, Y)

perm =PermutationImportance(m, random_state=1).fit(X, Y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2573 ± 0.0077,brand_cat
0.1033 ± 0.0075,feat_material_cat
0.0246 ± 0.0025,feat_gender_cat
0.0177 ± 0.0017,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0089 ± 0.0012,feat_metal type_cat
0.0030 ± 0.0004,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [0]:
!git add matrix_one/day5.ipynb